In [1]:
from lib.data_utils import *
from lib.model_utils import *
from lib.simulator import *

# Load data & models

## Data

In [2]:
df_train, df_train_y, df_optimization, df_optimization_y, df_test, df_test_y, complete_feature_set, features_model = load_model_training_data_template(train_sets = ['data/wyscout/csv/events/Italy.csv', 'data/wyscout/csv/events/Germany.csv', 'data/wyscout/csv/events/France.csv'], optimization_sets = ['data/wyscout/csv/events/Italy.csv',], test_sets = ['data/wyscout/csv/events/Spain.csv', 'data/wyscout/csv/events/England.csv'])

In [3]:
features = features_model['TYPE']
X_test = df_test[features].astype(float).values

In [4]:
sim = Simulator(
    model_type_path='models/lem/LEMv3_MODEL_TYPE_TORCH.pth',
    model_acc_path='models/lem/LEMv4_MODEL_ACC_TORCH.pth',
    model_data_path='models/lem/LEMv3_MODEL_DATA_TORCH.pth',
    device = 'cuda:0')

In [5]:
id_offset = 250 #Implement glob count
for i in range(200):
    feature_tensor = sim.simulate(X_test, store_full_sim=False)
    data = pd.DataFrame(feature_tensor.cpu())
    data['type_id'] = data[data.columns[:33]].idxmax(axis=1) + 1
    data = data.drop(data.columns[:33], axis=1)
    data.columns = ['period', 'minute', 'x', 'y', 'is_home_team', 'accurate', 'goal', 'home_score', 'away_score', 'type_id']
    data.home_score = (data.home_score * 10).astype(int)
    data.away_score = (data.away_score * 10).astype(int)
    data['type_name'] = data.type_id.map({v: k for k, v in SUBEVENT_TYPE_MAP.items()})
    data.to_csv(f'data/lem/lem_sim_{id_offset+i}.csv', index=False)

  0%|          | 4/2000 [00:02<16:05,  2.07it/s]